In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

2022-03-23 15:38:11.811916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 15:38:11.812012: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train'
#valid_path = '/home/vimalkumar/Documents/softco/CNN/CNN/test'

In [3]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


2022-03-23 15:39:26.810190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-23 15:39:26.810225: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-23 15:39:26.810242: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-03-23 15:39:26.810953: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [5]:
folders = glob('/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train/*')


In [6]:
x = Flatten()(inception.output)


In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   
                                  validation_split=0.15)

#test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 2,
                                                 class_mode = 'categorical',
                                                subset = 'training')

Found 1175 images belonging to 4 classes.


In [11]:
test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 2,
                                            class_mode = 'categorical',
                                           subset = 'validation')

Found 204 images belonging to 4 classes.


In [12]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  #steps_per_epoch=len(training_set),
  #validation_steps=len(test_set)
)

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-03-23 15:39:43.695073: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
588/588 [==============================] - 121s 200ms/step - loss: 5.4608 - accuracy: 0.7285 - val_loss: 6.5998 - val_accuracy: 0.7500
Epoch 2/25
588/588 [==============================] - 111s 188ms/step - loss: 2.2021 - accuracy: 0.8783 - val_loss: 4.8023 - val_accuracy: 0.7892
Epoch 3/25
588/588 [==============================] - 165s 280ms/step - loss: 1.1130 - accuracy: 0.9311 - val_loss: 10.3768 - val_accuracy: 0.6520
Epoch 4/25
588/588 [==============================] - 130s 221ms/step - loss: 0.9742 - accuracy: 0.9353 - val_loss: 4.6860 - val_accuracy: 0.8235
Epoch 5/25
588/588 [==============================] - 101s 172ms/step - loss: 1.0733 - accuracy: 0.9455 - val_loss: 7.1766 - val_accuracy: 0.7451
Epoch 6/25
588/588 [==============================] - 98s 167ms/step - loss: 0.7841 - accuracy: 0.9677 - val_loss: 5.6065 - val_accuracy: 0.8039
Epoch 7/25
588/588 [==============================] - 94s 160ms/step - loss: 1.0295 - accuracy: 0.9626 - val_loss: 5.4555 - 

In [13]:
model.save('stewart_rotation_inception_model.h5')

In [14]:
import os
import keras
from keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from glob import glob

In [15]:
model = keras.models.load_model('stewart_rotation_inception_model.h5')


In [16]:
train_path = '/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train'
folders = glob('/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train/*')

keys = []
values = []
for key in range(len(folders)):
    #print(key)
    keys.append(key)
for path in os.listdir(train_path):
    #print(path)
    values.append(path)
dic = dict(list(zip(keys,sorted(values))))
print(dic)

{0: '0 degree', 1: '180 degree', 2: '270 degree', 3: '90 degree'}


In [23]:
path = '/home/vimalkumar/Downloads/stewart degrees test/test/0 test'  



cnt = 1
for i in os.listdir(path):
    print(cnt , '  ',i)
    img = image.load_img(os.path.join(path,i), 
                         target_size=(224,224))
#     plt.imshow(img)
#     plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    #print(a)
    print('Rotation angle : ',dic[a[0]])

1    b17_2.jpeg
Rotation angle :  0 degree
1    b17_13.jpeg
Rotation angle :  180 degree
1    b17_33.jpeg
Rotation angle :  0 degree
1    b17_11.jpeg
Rotation angle :  0 degree
1    b17_18.jpeg
Rotation angle :  0 degree
1    b17_89.jpeg
Rotation angle :  0 degree
1    b17_19.jpeg
Rotation angle :  0 degree
1    b17_31.jpeg
Rotation angle :  180 degree
1    b17_87.jpeg
Rotation angle :  0 degree
1    b17_84.jpeg
Rotation angle :  180 degree
1    b17_24.jpeg
Rotation angle :  180 degree
1    b17_86.jpeg
Rotation angle :  0 degree
1    b17_35.jpeg
Rotation angle :  0 degree
1    b17_1.jpeg
Rotation angle :  0 degree
1    b17_8.jpeg
Rotation angle :  0 degree
1    b17_9.jpeg
Rotation angle :  0 degree
1    b17_3.jpeg
Rotation angle :  0 degree
1    b17_27.jpeg
Rotation angle :  0 degree
1    b17_34.jpeg
Rotation angle :  0 degree
1    b17_85.jpeg
Rotation angle :  0 degree
1    b17_6.jpeg
Rotation angle :  0 degree
1    b17_10.jpeg
Rotation angle :  0 degree
1    b17_26.jpeg
Rotation angl